<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_overlap_80_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    694      0 --:--:-- --:--:-- --:--:--   694
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_80.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,2,heel pain
3,2,heel pain
4,2,right heel became painful


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.855447,4.204043,0.219675,03:50


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.979640,3.711450,0.293431,03:44


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.705868,3.679305,0.299298,03:46


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.677823,3.656944,0.301626,03:53


In [0]:
learn.save_encoder('AskAPatient_overlap_80_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('AskAPatient_overlap_80_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.837033,3.471375,0.426035,01:14


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.872308,2.992386,0.482840,01:30


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.367980,2.635188,0.551479,03:34


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.915586,2.311023,0.611834,03:39
1,1.687151,2.179105,0.654438,03:31


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.585314,2.028065,0.655621,03:43
1,1.441709,1.873377,0.686391,03:53


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.338618,1.793298,0.704142,03:34
1,1.185531,1.704057,0.712426,03:48


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.123259,1.653165,0.714793,03:42
1,1.042153,1.580295,0.727811,03:39


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.005342,1.569300,0.736095,03:49
1,0.912023,1.508993,0.740828,03:39


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.871979,1.448879,0.757396,03:44
1,0.879716,1.469465,0.747929,03:49
2,0.759946,1.432290,0.764497,03:49
3,0.693112,1.426327,0.765680,03:45


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.706339,1.502252,0.760947,04:00
1,0.742741,1.402334,0.764497,03:41
2,0.649504,1.363746,0.771598,03:34
3,0.590928,1.325431,0.778698,04:08


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.565417,1.381618,0.768047,03:57
1,0.632684,1.430697,0.769231,03:42
2,0.589456,1.390058,0.775148,03:49
3,0.532687,1.362933,0.777515,03:49


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.530852,1.373046,0.778698,04:03
1,0.585313,1.377029,0.773965,03:51
2,0.596022,1.387156,0.785799,03:47
3,0.494466,1.379115,0.785799,04:02


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.501731,1.416571,0.781065,03:48
1,0.569516,1.382547,0.779882,03:44
2,0.507674,1.433519,0.777515,03:54
3,0.459213,1.354898,0.782248,04:00


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.521575,1.396508,0.783432,03:54
1,0.527994,1.351327,0.786982,03:47
2,0.491530,1.371657,0.789349,03:50
3,0.472078,1.430688,0.790533,03:57


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.461445,1.448060,0.789349,04:02
1,0.491969,1.480519,0.775148,04:02
2,0.499638,1.423225,0.788166,03:38
3,0.464061,1.389407,0.788166,03:42


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.469933,1.472776,0.784615,03:53
1,0.490892,1.439897,0.791716,03:47
2,0.474177,1.413972,0.791716,03:36
3,0.420462,1.397062,0.790533,04:10


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.468497,1.566244,0.782248,03:36
1,0.487739,1.432299,0.782248,03:47
2,0.460086,1.369009,0.794083,03:53
3,0.429596,1.413922,0.795266,03:32


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.442990,1.520327,0.791716,03:45
1,0.448446,1.436985,0.786982,03:55
2,0.415354,1.470024,0.797633,03:53
3,0.395810,1.432135,0.796450,03:43


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.441490,1.526049,0.788166,03:50
1,0.463082,1.443624,0.802367,03:38
2,0.456733,1.479303,0.795266,03:42
3,0.401101,1.462993,0.789349,04:06


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.429717,1.441165,0.795266,03:51
1,0.467838,1.451532,0.785799,03:40
2,0.436461,1.392701,0.797633,03:51
3,0.427609,1.432850,0.792899,04:08


In [0]:
# save the best model

learn.save_encoder('AskAPatient_overlap_80')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

3051
3051
[416, 2, 2, 2, 2, 2, 66, 4, 308, 334, 584, 5, 48, 48, 48, 48, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 269, 1035, 48, 8, 500, 10, 98, 123, 16, 16, 1035, 5, 71, 20, 21, 21, 964, 1035, 610, 23, 25, 27, 27, 27, 27, 1028, 27, 29, 29, 30, 30, 864, 30, 31, 584, 584, 584, 584, 584, 584, 32, 32, 34, 34, 34, 964, 704, 289, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 704, 37, 37, 37, 37, 37, 289, 289, 704, 289, 289, 704, 37, 614, 190, 218, 39, 432, 515, 42, 42, 44, 44, 44, 45, 48, 256, 1031, 1031, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 78, 52, 1032, 1032, 1032, 852, 852, 852, 852, 852, 852, 852, 852, 57, 58, 58, 59, 60, 60, 427, 61, 62, 62, 62, 62, 65, 1031, 69, 70, 70, 70, 70, 717, 776, 71, 71, 71, 71, 71, 71, 71, 491, 75, 76, 76, 76, 76, 289, 77, 289, 753, 122, 614, 614, 353, 353, 1031, 79, 614, 79, 260, 260, 260, 260, 260, 260, 260

In [50]:
correct_num = 0

for i in range(0, len(predict_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

1952
0.6397902327105867
